In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

# index 생성

In [ ]:
# ! pip install pinecone

   ---------------------------------------- 0.0/587.6 kB ? eta -:--:--
   ---------------------------------------- 587.6/587.6 kB 20.3 MB/s  0:00:00

  Attempting uninstall: packaging

    Found existing installation: packaging 25.0

    Uninstalling packaging-25.0:

      Successfully uninstalled packaging-25.0

   -------------------- ------------------- 2/4 [pinecone-plugin-assistant]
   ------------------------------ --------- 3/4 [pinecone]
   ------------------------------ --------- 3/4 [pinecone]
   ------------------------------ --------- 3/4 [pinecone]
   ------------------------------ --------- 3/4 [pinecone]
   ---------------------------------------- 4/4 [pinecone]



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-upstage 0.7.3 requires tokenizers<0.21.0,>=0.20.0, but you have tokenizers 0.22.0 which is incompatible.


In [8]:
# Create client
from pinecone import Pinecone

pinecone_client = Pinecone(api_key=PINECONE_API_KEY)

In [9]:
pinecone_client

In [ ]:
from pinecone import ServerlessSpec

pinecone_client.create_index(
    name='embedding-3d',
    dimension=3,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

# 2번 실행에서 에러뜸
# index 만들어짐

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-04', 'x-cloud-trace-context': '3eb9e6d829a2b6b358f10c8c316aa850', 'date': 'Mon, 08 Sep 2025 07:37:33 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [11]:
index = pinecone_client.Index('embedding-3d')
index

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [15]:
for ids in index.list(namespace='embedding-3d-ns1'):
    print(ids) 

# pinecone data 형식
[{'id': 'vec1', 'values': [1.0, 1.5, 2.0], 'metadata': {'genre': 'drama'}},\
{'id': 'vec2', 'values': [2.0, 1.5, 0.5], 'metadata': {'genre': 'action'}},\
{'id': 'vec3', 'values': [0.1, 0.3, 0.5], 'metadata': {'genre': 'drama'}},\
{'id': 'vec4', 'values': [1.0, 2.5, 3.5], 'metadata': {'genre': 'action'}},\
{'id': 'vec5', 'values': [3.0, 1.2, 1.3], 'metadata': {'genre': 'action'}},\
{'id': 'vec6', 'values': [0.3, 1.1, 2.5], 'metadata': {'genre': 'drama'}},\
]

In [16]:
index.upsert(
    vectors = [
        {'id': 'vec1', 'values': [1.0, 1.5, 2.0], 'metadata': {'genre': 'drama'}},
        {'id': 'vec2', 'values': [2.0, 1.5, 0.5], 'metadata': {'genre': 'action'}},
        {'id': 'vec3', 'values': [0.1, 0.3, 0.5], 'metadata': {'genre': 'drama'}},
        {'id': 'vec4', 'values': [1.0, 2.5, 3.5], 'metadata': {'genre': 'action'}},
        {'id': 'vec5', 'values': [3.0, 1.2, 1.3], 'metadata': {'genre': 'action'}},
        {'id': 'vec6', 'values': [0.3, 1.1, 2.5], 'metadata': {'genre': 'drama'}},
    ],
    namespace='embedding-3d-ns1'
    ) # update + insert => 없으면 insert하고 있으면 update

{'upserted_count': 6}

In [17]:
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'embedding-3d-ns1': {'vector_count': 6}},
 'total_vector_count': 6,
 'vector_type': 'dense'}

# id list

In [18]:
for ids in index.list(namespace='embedding-3d-ns1'):
    print(ids) 

['vec1', 'vec2', 'vec3', 'vec4', 'vec5', 'vec6']


# query

In [20]:
response = index.query(
    namespace='embedding-3d-ns1',
    vector=[0.1, 0.4, 0.7],
    top_k=3,
    include_values=True,
    include_metadata=True,
    filter={'genre': {'$eq': 'drama'}}
)

response

{'matches': [{'id': 'vec3',
              'metadata': {'genre': 'drama'},
              'score': 1.00008953,
              'values': [0.1, 0.3, 0.5]},
             {'id': 'vec6',
              'metadata': {'genre': 'drama'},
              'score': 0.99550879,
              'values': [0.3, 1.1, 2.5]},
             {'id': 'vec1',
              'metadata': {'genre': 'drama'},
              'score': 0.961182,
              'values': [1.0, 1.5, 2.0]}],
 'namespace': 'embedding-3d-ns1',
 'usage': {'read_units': 1}}